In [65]:
import datetime
import ollama
import pandas as pd
import requests
import time
from typing import Dict, List

In [89]:
SLEEPER_REST_API = "https://api.sleeper.com"
SLEEPER_GRAPHQL_API = "https://sleeper.com/graphql"


def get_player_reports(player_id: str, limit: int) -> List[Dict]:
    operation_name = "get_player_news"
    kwargs = {
        "operation_name": operation_name,
        "player_id": player_id,
        "limit": limit,
    }
    query = """
        query {operation_name} {{
            get_player_news(sport: "nfl", player_id: "{player_id}", limit: {limit}){{
                metadata
                player_id
                published
                source
                source_key
                sport
            }}
        }}
    """.format(**kwargs)
    body = {
        "operationName": operation_name,
        "query": query,
        "variables": {}
    }
    time.sleep(0.1)
    req = requests.post(SLEEPER_GRAPHQL_API, json=body)
    res = req.json()
    reports = res.get("data", {}).get(operation_name, [])
    return reports


def transform_reports(raw_reports: List[Dict]) -> List[Dict]:
    return [
        {
            "report_id": r.get("source") + "_" + r.get("source_key"),
            "player_id": r.get("player_id"),
            "source": r.get("source"),
            "category": "retrospective" if r.get("metadata", {}).get("analysis") is not None else "prospective",
            "published_at": r.get("published"),
            "title": r.get("metadata", {}).get("title"),
            "description": r.get("metadata", {}).get("description"),
            "analysis": r.get("metadata", {}).get("analysis"),
        }
        for r in raw_reports
    ]


def get_player_week_stats(player_id: str, season: int) -> List[Dict]:
    url = f"{SLEEPER_REST_API}/stats/nfl/player/{player_id}?season_type=regular&season={season}&grouping=week"
    time.sleep(0.1)
    req = requests.get(url)
    res = req.json()
    raw = res.values()
    week_stats = list(filter(lambda v: v is not None, raw))
    return week_stats


def transform_week_stats(raw_week_stats: List[Dict]) -> List[Dict]:
    return [
        {
            "stat_id": r.get("season") + "_" + str(r.get("week")) + "_" + r.get("player_id"),
            "week_id": r.get("season") + "_" + str(r.get("week")),
            "player_id": r.get("player_id"),
            "game_id": r.get("game_id"),
            "season": r.get("season"),
            "week": r.get("week"),
            "team": r.get("team"),
            "opponent": r.get("opponent"),
            "played_at": int(datetime.datetime.strptime(r.get("date"), "%Y-%m-%d").timestamp() * 1000),
            "rushing_attempts": int(r.get("stats", {}).get("rush_att", 0)),
            "rushing_yards": int(r.get("stats", {}).get("rush_yd", 0)),
            "receiving_targets": int(r.get("stats", {}).get("rec_tgt", 0)),
            "receiving_catches": int(r.get("stats", {}).get("rec", 0)),
            "receiving_yards": int(r.get("stats", {}).get("rec_yd", 0))
            
        }
        for r in raw_week_stats
    ]

In [90]:
raw_reports = get_player_reports(player_id="4147", limit=10)
reports = transform_reports(raw_reports)
df_reports = pd.DataFrame(reports).query("category == 'prospective'")

In [91]:
raw_stats = get_player_week_stats(player_id="4147", season=2025)
stats = transform_week_stats(raw_stats)
df_stats = pd.DataFrame(stats)

In [92]:
df_reports.head()

,report_id,player_id,source,category,published_at,title,description,analysis
0,rotoballer_212390,4147,rotoballer,prospective,1767121900000,Samaje Perine a Top Waiver Priority for Week 18,Cincinnati Bengals running back Samaje Perine ...,None
3,rotoballer_212107,4147,rotoballer,prospective,1766852734000,Samaje Perine in the Flex Conversation in Week...,Cincinnati Bengals running back Samaje Perine ...,None
4,rotoballer_211920,4147,rotoballer,prospective,1766693558000,Samaje Perine Working Toward Fantasy Viability,Cincinnati Bengals running back Samaje Perine ...,None
5,rotoballer_211695,4147,rotoballer,prospective,1766420973000,Samaje Perine Offers Standalone Value with Bui...,Cincinnati Bengals running back Samaje Perine ...,None
8,rotoballer_211057,4147,rotoballer,prospective,1765903055000,Samaje Perine a Priority Handcuff Option With ...,Since returning in Week 13 from an ankle injur...,None


In [93]:
df_stats.head()

,stat_id,week_id,player_id,game_id,season,week,team,opponent,played_at,rushing_attempts,rushing_yards,receiving_targets,receiving_catches,receiving_yards
0,2025_1_4147,2025_1,4147,202510108,2025,1,CIN,CLE,1757221200000,0,0,2,2,6
1,2025_13_4147,2025_13,4147,202511303,2025,13,CIN,BAL,1764223200000,14,39,2,2,15
2,2025_14_4147,2025_14,4147,202511404,2025,14,CIN,BUF,1765087200000,6,31,1,1,7
3,2025_15_4147,2025_15,4147,202511507,2025,15,CIN,BAL,1765692000000,14,42,1,1,1
4,2025_16_4147,2025_16,4147,202511619,2025,16,CIN,MIA,1766296800000,7,25,3,2,16
